In [1]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance ## Feature Importance를 불러오기 위함
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.preprocessing import RobustScaler
import warnings
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [128]:
data = pd.read_csv("eco jeju/final_final_train_0913.csv")

In [129]:
data.drop(columns=['Unnamed: 0'],inplace=True)

In [130]:
data.sort_values(by=['base_date'],inplace=True)

In [131]:
data.reset_index(inplace=True)

In [132]:
data.drop(columns=['index'],inplace=True)

In [133]:
data

,base_date,emd_nm,em_g,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop
0,2018-01-01,전체,85755000,679922,535888.570169,3.786578e+09,20891720.0,1.602637e+09,108983.0,1057.0,40848.0,32512,7.210523e+06,830189.1305,54466.952499,2215.223209
1,2018-01-01,오라동,2128300,13829,12662.099974,9.421414e+07,436990.0,4.826442e+07,2652.0,25.0,1105.0,818,2.505763e+05,22012.2070,1377.472687,41.620260
2,2018-01-01,외도동,3476650,21105,8902.768868,1.373002e+08,359412.0,6.363159e+07,4313.0,15.0,1979.0,1425,1.126928e+05,10341.1722,839.760964,39.834595
3,2018-01-01,용담1동,1487100,7887,6631.149998,5.763687e+07,476270.0,3.462198e+07,1317.0,20.0,565.0,532,1.019396e+05,9363.6225,601.809124,28.697845
4,2018-01-01,용담2동,2581850,15916,10504.580161,1.051404e+08,291163.0,4.387306e+07,3173.0,20.0,1257.0,1052,2.892350e+05,44610.5275,1609.868207,139.279035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50881,2021-06-30,성산읍,3380000,16972,13809.659196,8.855328e+07,2250850.0,2.500685e+07,1720.0,101.0,624.0,737,2.503109e+05,24379.9565,1612.380993,29.272263
50882,2021-06-30,송산동,1090000,4510,1787.890608,3.939310e+07,968778.0,1.469781e+07,738.0,40.0,302.0,290,4.834354e+04,3387.0801,230.594645,14.582817
50883,2021-06-30,아라동,3565000,39396,42414.120501,2.900830e+08,24297973.0,8.831433e+07,9149.0,1066.0,2590.0,2001,3.830194e+05,121879.6674,4231.797354,41.029042
50884,2021-06-30,도두동,597050,3403,1183.279682,4.068692e+07,1303776.0,1.502919e+07,758.0,50.0,246.0,292,8.122313e+04,4625.5674,349.232990,11.704028


In [216]:
X_features = data.loc[:, ['base_date','youth_pop', 'total_frgn_pop', 'total_pop', 
                                'jeju_resd_pop', 'card_use_amt', 'delivery_amt',
                                'market_amt', 'card_use_cnt','market_cnt','em_cnt','visit_pop','work_pop']]
y_label = data.loc[:, 'em_g']

In [87]:
from sklearn.preprocessing import OrdinalEncoder
order=OrdinalEncoder()
X_features['base_date']=order.fit_transform(X_features['base_date'].values.reshape(-1,1))

In [88]:
X_features.shape

(50886, 13)

In [217]:
#6월1일까지 train
X_train = X_features.iloc[:49638]
X_valid = X_features.iloc[49638:]
y_train = y_label.iloc[:49638]
y_valid = y_label.loc[49638:]
print(X_train.shape, X_valid.shape,y_train.shape,y_valid.shape)

(49638, 13) (1248, 13) (49638,) (1248,)


In [90]:
scaler = RobustScaler()

X_train[['youth_pop', 'total_frgn_pop', 'total_pop', 
                                'jeju_resd_pop', 'card_use_amt', 'delivery_amt',
                                'market_amt', 'card_use_cnt','market_cnt','em_cnt','visit_pop','work_pop']] = scaler.fit_transform(X_train[['youth_pop', 'total_frgn_pop', 'total_pop', 
                                'jeju_resd_pop', 'card_use_amt', 'delivery_amt',
                                'market_amt', 'card_use_cnt','market_cnt','em_cnt','visit_pop','work_pop']])
X_valid[['youth_pop', 'total_frgn_pop', 'total_pop', 
                                'jeju_resd_pop', 'card_use_amt', 'delivery_amt',
                                'market_amt', 'card_use_cnt','market_cnt','em_cnt','visit_pop','work_pop']] = scaler.transform(X_valid[['youth_pop', 'total_frgn_pop', 'total_pop', 
                                'jeju_resd_pop', 'card_use_amt', 'delivery_amt',
                                'market_amt', 'card_use_cnt','market_cnt','em_cnt','visit_pop','work_pop']])

X_scaled_train = pd.DataFrame(X_train , index = X_train.index , columns = X_train.columns)
X_scaled_valid = pd.DataFrame(X_valid , index = X_valid.index , columns = X_valid.columns)

In [91]:
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=10,
                       learning_rate=0.01, 
                       n_estimators=8000,
                       max_bin=200, 
                       verbose=-1,
                       random_state=42)

In [92]:
lightgbm.fit(X_train,y_train)

LGBMRegressor(learning_rate=0.01, max_bin=200, n_estimators=8000, num_leaves=10,
              objective='regression', random_state=42, verbose=-1)

In [93]:
pred=lightgbm.predict(X_valid)

In [95]:
predict = pd.DataFrame(pred, columns=['predict'])
predict

,predict
0,2.980409e+06
1,1.021409e+06
2,5.851324e+06
3,3.212802e+06
4,1.422742e+06
...,...
1243,2.538348e+06
1244,8.897632e+05
1245,3.531024e+06
1246,8.304210e+05


In [96]:
predict['y']=np.array(y_valid)

In [97]:
predict['predict']=predict['predict'].astype('int')

In [98]:
predict

,predict,y
0,2980409,2509350
1,1021409,928050
2,5851324,5635900
3,3212801,3296450
4,1422741,1582000
...,...,...
1243,2538347,3380000
1244,889763,1090000
1245,3531023,3565000
1246,830421,597050


In [94]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print(np.sqrt(mean_squared_error(y_valid,pred)))
print(mean_absolute_error(y_valid,pred))
print(lightgbm.score(X_train,y_train))
print(lightgbm.score(X_valid,y_valid))

638621.4609382043
221641.7179955904
0.9991535031896661
0.9978685614835527
